In [11]:
import pandas as pd
import os
 
# Diretório onde os XLSX estão localizados
caminho_xlsx = 'C:\\AFL_GABRIEL\\csv_parser\\DATA\\gabarito\\2299x1010'  
caminho_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\DATA_PARQUET\\gabarito\\2299x1010'  
 
# Criar o diretório de saída, se não existir
if not os.path.exists(caminho_parquet):
    os.makedirs(caminho_parquet)
 
# Lista para armazenar DataFrames das páginas
dfs_paginas = {}
 
# Iterar pelos arquivos XLSX no diretório
for arquivo in os.listdir(caminho_xlsx):
    if arquivo.endswith('.xlsx'):
        # Caminho completo do arquivo XLSX
        caminho_xlsx = os.path.join(caminho_xlsx, arquivo)
       
        try:
            # Lê todas as planilhas do arquivo XLSX em um dicionário
            sheets_dict = pd.read_excel(caminho_xlsx, sheet_name=None)
        except Exception as e:
            print(f'Erro ao ler {arquivo}: {e}')
            continue
       
        # Extrai o nome base antes da parte de paginação, se houver
        nome_base = arquivo.split('_pagina')[0]
       
        # Se ainda não tiver uma entrada para este nome_base, cria
        if nome_base not in dfs_paginas:
            dfs_paginas[nome_base] = []
       
        # Itera por todas as páginas do arquivo e adiciona os DataFrames
        for nome_sheet, df in sheets_dict.items():
            if not df.empty:
                dfs_paginas[nome_base].append(df)
                print(f'Lido: {arquivo} - Página: {nome_sheet} com {df.shape[0]} linhas')
 
# Processar e unir os DataFrames por nome_base
for nome_base, lista_dfs in dfs_paginas.items():
    # Se houver mais de uma página, concatena os DataFrames
    if len(lista_dfs) > 1:
        df_final = pd.concat(lista_dfs, ignore_index=True)
        print(f'{nome_base} - Páginas unidas com {df_final.shape[0]} linhas')
    elif lista_dfs:
        df_final = lista_dfs[0]
    else:
        print(f'{nome_base} - Nenhum dado encontrado')
        continue
   
    # Caminho para salvar o arquivo Parquet
    caminho_parquet = os.path.join(caminho_parquet, f'{nome_base}.parquet')
   
    # Tenta salvar em formato Parquet
    try:
        df_final.to_parquet(caminho_parquet, index=False)
        print(f'Salvo: {caminho_parquet}')
    except Exception as e:
        print(f'Erro ao salvar {nome_base}.parquet: {e}')
 
print('Conversão concluída!')

Lido: 2299 x 1010.xlsx - Página: Parte 1 com 37568 linhas
Salvo: C:\AFL_GABRIEL\csv_parser\DATA_PARQUET\gabarito\2299x1010\2299 x 1010.xlsx.parquet
Conversão concluída!


In [15]:
import pandas as pd
import pyperclip

# Caminho do arquivo Parquet
caminho_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\DATA_PARQUET\\gabarito\\1200x2200'

# Lista de palavras-chave para conversão e substituição
palavras_chave = ["Vlr", "Valor", "vlr", "valor"]  # Modifique conforme necessário

# Ler o arquivo Parquet
df = pd.read_parquet(caminho_parquet)

# Informações do DataFrame
num_linhas, num_colunas = df.shape  # Obter número de linhas e colunas

# Criar a string com os resultados
resultado = f"🔹 Total de Linhas: {num_linhas}\n🔹 Total de Colunas: {num_colunas}\n\n📊 Detalhes das Colunas:\n"

# Percorre todas as colunas e adiciona detalhes
for coluna in df.columns:
    linhas_preenchidas = df[coluna].count()  # Conta valores não nulos
    
    # Verifica se a coluna está na lista de palavras-chave
    if any(palavra.lower() in coluna.lower() for palavra in palavras_chave):
            # Converte para float, ignorando erros caso tenha valores inválidos
            df[coluna] = (
                df[coluna].astype(str)
                .str.replace('.', '', regex=False)      # Remove milhar
                .str.replace(',', '.', regex=False)     # Converte decimal
                .str.extract(r'(\d+\.?\d*)')[0]          # Extrai número
                .astype(float)
            )
            # Soma
            soma_valores = df[coluna].sum(skipna=True)

            # Exibição em formato brasileiro
            valor_formatado = f"{soma_valores:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

            resultado += f"\n📌 Coluna: {coluna} (🔄 Convertida para Float e Substituição de '.' por ',')\n"
            resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"
            resultado += f"   🔹 Total Somado: {soma_valores:,.2f}\n"
    else:
        resultado += f"\n📌 Coluna: {coluna}\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"

# Copiar para a área de transferência
pyperclip.copy(resultado)

# Exibir os resultados
print("\n✅ Resultado copiado para a área de transferência!")



✅ Resultado copiado para a área de transferência!
